<a href="https://colab.research.google.com/github/wileyw/DeepLearningDemos/blob/master/NeuralArchitectureSearch/NeuralArchitectureSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Architecture Search

## EfficientNet

### What's the difference between MBConv1 versus MBConv6?
- MBConv1 expands the number of channels by a factor of 1.
- MBConv6 expands the number of channels by a factor of 6.

[MBConv6](https://machinethink.net/blog/mobile-architectures/)

## Do you need to do your own Neural Architecture Search?
- No, you can use transfer learning to apply an existing neural net architecture and retrain the network on your own dataset
- [AutoML and Neural Architecture Search](https://towardsdatascience.com/everything-you-need-to-know-about-automl-and-neural-architecture-search-8db1863682bf)


## Goals
1. Understand EfficientNet
2. Understand why EfficientNet is better than Mobilenet V3, could be a blog post
3. PyTorch EfficientDet
4. Run EfficientNet interence
5. Does EfficientNet produce a different network depending on the dataset?
6. How does MNas work?

# Talking Points
## 1. What are Inverted Residual Blocks?
- 1x1 Convolution Expands the Filter Dimensions --> 3x3 Depthwise Convolution --> 1x1 Separable Convolution Reduces the number of Filter Dimensions
- NOTE: The Blocks used for Skip Connections do not have non-linearities (e.g. activation functions)

# How to build EfficientNet
## Step 1.
Find the baseline EfficientNet network using Neural Architecture Search

## Step 2.
Scale up the baseline network using grid search

# Neural Architecture Search
## Tunable Parameters
- Depth, Width (Channels), Resolutions

In [0]:
!python3 -m pip install efficientnet_pytorch
!python3 -m pip install torchsummary

In [0]:
!git clone https://github.com/lukemelas/EfficientNet-PyTorch

In [0]:
!ls EfficientNet-PyTorch/examples
!ls EfficientNet-PyTorch/examples/simple/
!cp EfficientNet-PyTorch/examples/simple/img.jpg .
!cp EfficientNet-PyTorch/examples/simple/labels_map.txt .

In [0]:
import torch
import torchsummary
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_name('efficientnet-b0')

print(model)
torchsummary.summary(model, input_size=(3, 224, 224))

In [0]:
import json
from PIL import Image
import torch
from torchvision import transforms

from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b0')

# Preprocess image
tfms = transforms.Compose([transforms.Resize(224), transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])
img = tfms(Image.open('img.jpg')).unsqueeze(0)
print(img.shape) # torch.Size([1, 3, 224, 224])

# Load ImageNet class names
labels_map = json.load(open('labels_map.txt'))
labels_map = [labels_map[str(i)] for i in range(1000)]

# Classify
model.eval()
with torch.no_grad():
    outputs = model(img)

# Print predictions
print('-----')
for idx in torch.topk(outputs, k=5).indices.squeeze(0).tolist():
    prob = torch.softmax(outputs, dim=1)[0, idx].item()
    print('{label:<75} ({p:.2f}%)'.format(label=labels_map[idx], p=prob*100))

# Resources

* [EfficientNet Paper](https://arxiv.org/pdf/1905.11946.pdf)
* [EfficientDet Paper](https://arxiv.org/pdf/1911.09070.pdf)
* [Learning OpenCV EfficientNet](https://www.learnopencv.com/efficientnet-theory-code/)
* [Tensorflow EfficientNet Implementation](https://github.com/tensorflow/tpu/blob/master/models/official/efficientnet/efficientnet_model.py)
* [PyTorch Implementation](https://github.com/lukemelas/EfficientNet-PyTorch)
* [MnasFPN : Learning Latency-aware Pyramid Architecture
for Object Detection on Mobile Devices](https://arxiv.org/pdf/1912.01106.pdf)
* [Yolo V4](https://arxiv.org/pdf/2004.10934.pdf)
* [EffResNetComparison](https://colab.research.google.com/github/rwightman/pytorch-image-models/blob/master/notebooks/EffResNetComparison.ipynb#scrollTo=SKA-MF-yShDW)
